# Main Libraries

In [ ]:
# Install Libraries
!pip install psycopg2
!pip install psycopg2-binary
!pip install OTXv2
!pip install import-ipynb
!pip install haversine
!pip install geoip2

# Load libraries

In [ ]:
# Load libraries
import psycopg2
from OTXv2 import OTXv2, IndicatorTypes
import requests as req
import pandas as pd
from pandas import json_normalize
import json
import time
from datetime import datetime
import os
from bson import json_util, ObjectId
import geoip2.database
import haversine as hs
from ipaddress import ip_address

# PostgreSQL Connection Constants

In [ ]:
pgUser = ""
pgPass = ""
pgHost = ""
pgDB = ""
pgPort = ""

# Functions

In [ ]:
def getConnData(pgUser, pgPass, pgHost, pgDB, pgPort):
	qryConnData = '''
  SELECT 	c.creation_date, 
      c.public_ip, 
      c.country_id, 
      u.email
  FROM connections.connections c
  INNER JOIN connections.users u
    ON u.user_id = c.user_id
    '''
	pgConn = psycopg2.connect(database=pgDB, user=pgUser, password=pgPass, host=pgHost, port=pgPort)
	cur = pgConn.cursor()
	cur.execute(qryConnData)
	rows = cur.fetchall()
	
	dfConnData = pd.DataFrame(rows, columns=["creation_date", "public_ip", "country_id", "email"])
	
	return dfConnData


def getGeoData(ip, geoIPList):
	if ip_address(ip).is_private:
		geoData = { "geo_ip":ip,
			    "geo_country_name":"PRIVATE IP",
			    "geo_subdivisions":"", 
			    "geo_city_name":"",
			    "geo_lat":0.0,
			    "geo_lon":0.0,
			    "geo_iso_code":"",
			    "geo_postal_code":""}
	
	else:
		geodb_reader = geoip2.database.Reader('/content/drive/MyDrive/SANS_DFIR_2022/GeoLite2-City.mmdb')
		geoResponse = geodb_reader.city(ip)
	
		geoData = { "geo_ip":ip,
				    "geo_country_name":geoResponse.country.name,
				    "geo_subdivisions":geoResponse.subdivisions.most_specific.name, 
				    "geo_city_name":geoResponse.city.name,
				    "geo_lat":geoResponse.location.latitude,
				    "geo_lon":geoResponse.location.longitude,
				    "geo_iso_code":geoResponse.country.iso_code,
				    "geo_postal_code":geoResponse.postal.code}
	
	geoIPList.append(geoData)
	
	return geoIPList


def getIndicators(session, ip, otxList):
    headers = {"X-OTX-API-KEY": "GET_API_KEY_FROM_ALIENVAULT_OTX",
                   "Accept": "application/json"}
    
    otx_url = "https://otx.alienvault.com/api/v1/indicator/IPv4/%s/malware" % ip
    
    try:
        response = session.get(otx_url, headers=headers, timeout=3)
    
        public_ip = {"public_ip": ip}
        new_json = {}
        data_len = len(response.json()["data"])
        if data_len > 0:
            new_json = response.json()["data"][0]
            new_json.update(public_ip)
        else:
            new_json = {"datetime_int": "", "hash": "", "detections": "", "date": ""}
            new_json.update(public_ip)
    
    except Exception as e:
        public_ip = {"public_ip": ip}
        new_json = {"datetime_int": "", "hash": "", "detections": "", "date": ""}
        new_json.update(public_ip)
        pass
    
    otxList.append(new_json)
    
    return otxList


def checkEVA(session, src_email, evaList):
    url = "https://api.eva.pingutil.com/email"
    params = { "email": src_email }

    response = session.get(url, params=params)

    evaList.append(response.json()["data"])

    return evaList


def getOfficeNearestDistance(geo_lat, geo_lon, country_id, iso_country_id, dfOffices):
    lstDistance = []
    distance = 0

    # Filter only Offices available within the connection's source country
    dfOffices = dfOffices[(dfOffices.suc_country_id == country_id)]
    
    if country_id == iso_country_id:
        for index, row in dfOffices.iterrows():

            #try:
            suc_city = row["suc_city"] 
            suc_full_name = row["suc_full_name"]
            suc_full_address = row["suc_full_address"]
           
            geo_lat_lon = (float(geo_lat), float(geo_lon))

            if "," in str(row["suc_latitude"]): row["suc_latitude"] = str(row["suc_latitude"]).replace(",",".")
            if "," in str(row["suc_longitude"]): row["suc_longitude"] = str(row["suc_longitude"]).replace(",",".")

            suc_latitude = float(row["suc_latitude"])
            suc_longitude = float(row["suc_longitude"])
            suc_lat_lon = (suc_latitude, suc_longitude)

            distance = round(hs.haversine(geo_lat_lon, suc_lat_lon), 2)

            lstDistance.append(distance)

        if not lstDistance:
            lstDistance.append(distance)

    else:
        lstDistance.append(distance)

    nearest_suc_distance = min(lstDistance, key=lambda suc_data:distance)

    return nearest_suc_distance

# Generating Initial DataFrames from Sources

In [ ]:
# Getting Site connections data from CSV
dfConnData = pd.read_csv(r"/content/drive/MyDrive/SANS_DFIR_2022/connections.csv")
# Just get 10% of the rows (randomly)
dfConnData = dfConnData.sample(frac = 0.005)

# Getting Site connections data from DB
#dfConnData = getConnData(pgUser, pgPass, pgHost, pgDB, pgPort)

dfConnData

In [ ]:
# Getting Site offices data
dfOffices = pd.read_csv(r"/content/drive/MyDrive/SANS_DFIR_2022/offices.csv")
dfOffices

In [ ]:
# Getting Countries ISO codes
dfISOCountries = pd.read_csv(r"/content/drive/MyDrive/SANS_DFIR_2022/iso3countries_v2.csv")

# Select columns iso_country_id, iso2 for later joining
dfISOCountries = dfISOCountries[["iso_country_id", "iso2"]]
dfISOCountries

# Obtain UNIQUE IPs and Email addresses for processing

In [ ]:
# Obtanin IP list from dataframe
lstIP = dfConnData['public_ip'].tolist()
# Removing duplicate IPs
lstIP = set(lstIP)

In [ ]:
# Obtanin Email list from dataframe
lstMails = dfConnData['email'].tolist()
# Removing duplicate Emails
lstMails = set(lstMails)

# Obtain and process Geo Data

In [ ]:
# Init Geoloc data list
geoIPList = []

# Get Geolocation data for each IP
[ getGeoData(ip, geoIPList) for ip in lstIP ]

# Generate Dataframe with the Geoloc data
dfGeoIPData = pd.DataFrame(geoIPList)
dfGeoIPData

# START API REQUESTS CALLS

## Obtain and process Threat-intel data from AlienVault OTX

In [ ]:
otxList = []

# Get AlienVault OTX data for each IP
sessionOTX = req.session()
[ getIndicators(sessionOTX, ip, otxList) for ip in lstIP ]

# Generate Dataframe with the AlienVault OTX data
dfOTX = pd.DataFrame(otxList, columns=["hash", "detections", "date", "public_ip"])
dfOTX

## Obtain and process Email address data from EVA

In [ ]:
# Traditional way (session requests)
evaList = []

# Get EVA data for each IP
sessionEVA = req.session()
[ checkEVA(sessionEVA, em, evaList) for em in lstMails ]

# Generate Dataframe with EVA data
dfEVA = pd.DataFrame(evaList, columns=["email_address","domain","valid_syntax",
                                       "disposable","webmail","deliverable",
                                       "catch_all","gibberish","spam"])
dfEVA

# Merge Connections DF with GeoLocation DF

In [ ]:
# Adding Geolocation data to Connections dataframe
dfFullData = pd.merge(dfConnData, dfGeoIPData, left_on='public_ip', right_on='geo_ip')

# Merging and adding useful ISO data to Geo dataframe for later use.
# "country_id" (ID entered by the user) VS. "iso_country_id" (actual ID based on IP Geo location)
dfFullData = pd.merge(dfFullData, dfISOCountries, left_on="geo_iso_code", right_on="iso2")
dfFullData

# Merge Connections DF with AlienVault DF

In [ ]:
# Adding AlientVault OTX data to Connections dataframe
dfFullData = pd.merge(dfFullData, dfOTX, how="left", on="public_ip")
dfFullData

# Merge Connections DF with EVA DF

In [ ]:
# Adding EVA data to Connections dataframe
dfFullData = pd.merge(dfFullData, dfEVA, left_on="email", right_on="email_address")
dfFullData

# Nearest Office

In [ ]:
# Adding Nearest Office distance from IP GeoLocation to Connections dataFrame
dfFullData['nearest_office_data'] = dfFullData.apply(
    lambda row: getOfficeNearestDistance(row['geo_lat'], row['geo_lon'], 
                                         row['country_id'], row['iso_country_id'], 
                                         dfOffices), axis=1)
dfFullData

# Filtering Results (optional)

In [ ]:
# Filter DF
dfFullDataFiltered = dfFullData.query("hash != ''")
dfFullData

# Filtering using variables
country_to_search = 'Turkey'
dfFullDataFilteredByCountry = dfFullDataFiltered.query("geo_country_name == @country_to_search")
dfFullDataFilteredByCountry

# Plotting Results

In [ ]:
# Install GeoPandas libraries
!pip install geopandas

In [ ]:
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

geometry = [Point(xy) for xy in zip(dfFullData['geo_lon'], dfFullData['geo_lat'])]
gdf = GeoDataFrame(dfFullData, geometry=geometry)   

#World Map (Cities)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(15, 7), color="grey"), marker='o', color='red', markersize=10);

In [ ]:
# Countries Chart
dfFullData.loc[(dfFullData.geo_country_name == 'United States of America'),'geo_country_name']='USA'
dfFullData.groupby(['geo_country_name']).count()['creation_date'].plot(title="Requests by Country", 
                                                                       kind="barh", 
                                                                       figsize=(11,5), 
                                                                       colormap='Paired')

# Send to Google Sheets

In [ ]:
# Install pygsheets to interact with Google Sheets
!pip install pygsheets
!pip install --upgrade -q pygsheets

In [ ]:
# Authenticate from Colab  
import google.auth
from google.colab import auth
auth.authenticate_user()

# Use current credentials to connect to Google Sheets
import pygsheets
credentials, _ = google.auth.default()
gc = pygsheets.client.Client(credentials)

In [ ]:
sh = gc.open('SANSDFIR2022')
wks = sh.worksheet_by_title('Report')
wks.clear()
wks.set_dataframe(dfFullData, (1, 1))